In [1]:
## Examples for generation of PCFGs and plots of useful quantities.

In [2]:
import sys
sys.path.append('../syntheticpcfg')


In [3]:
import pcfg
pcfg1 = pcfg.load_pcfg_from_file("../data/manual/example1.pcfg")

In [4]:
sampler1 = pcfg.Sampler(pcfg1)

In [13]:
import utility
t1 = sampler1.sample_tree()
s1 = utility.collect_yield(t1)
print(t1,s1)

('S', ('A', 'a'), ('B', 'b')) ['a', 'b']


In [14]:
import inside


inside1 = inside.InsideComputation(pcfg1)


inside1.inside_log_probability(s1)

-0.6931471805599453

In [16]:
import pcfgfactory

pcfgfactory1 = pcfgfactory.PCFGFactory()
pcfgfactory1.lexical_distribution = pcfgfactory.LexicalPitmanYor()

upcfg = pcfgfactory1.sample()

5 100 40 100


AttributeError: 'PCFGFactory' object has no attribute 'length_weights'